In [269]:
import pandas as pd
import random
from itertools import product

# Load the data
moderator_data = pd.read_excel("./Scoring/moderator_scored.xlsx")

# Moderator Matching Algorithm
**Purpose:**
- Increase Revenue: By matching the best reviewers with the top priority ads, we ensure that high-quality ads get the attention they deserve, leading to better user experience and potentially higher revenue

- Improve Resource Utilization: By dynamically allocating tasks based on moderator capacity and expertise, we aim to increase the utilization of ad moderators by at most 10%

In [270]:
ads_dataset = pd.read_excel("./Scoring/ads_with_score.xlsx", nrows = 30000)

# Gurobipy Optimization Model (Moderators)
1. **Decision Variables**
- For each ad and each moderator, a binary decision variable is created to represent whether the ad is assigned to the moderator (1) or not (0)

2. **Objective Function**
- The first objective function to minimize the difference between ad's score and moderator's score, this ensures that the top ads get matched with the best moderators
- The second objective function is with regards to the confidence of the ad, which is defined as how confident our model is in identifying whether there is a possible violation in the ad
    - If confidence is high, the ad is "easier" to moderator as it is easy to spot whether there is a violation. Hence the ad will be allocated to moderators that have high productivity so the ad can be cleared quickly, and low accuracy as a tradeoff
    - For ads with low confidence, it will be allocated to moderators that have high accuracy so that violations can be correctly identified, with the tradeoff being low productivity
- Combining these two objective functions, the overall objective function is as follows:
$$
\text{Minimize: } \sum_{\text{ad, mod}} \left| \beta_1 (ad\_score - moderator\_score) + \beta_2 (ad\_confidence - normalized\_productivity + normalized\_accuracy) \right|
$$

The coefficients $\beta_1$ and $\beta_2$ in the objective function represent weights that determine the importance of matching `ad_score` with `moderator_score` and `ad_confidence` with `normalized_productivity` and `normalized_accuracy`, respectively. Ideally, these coefficients should be derived from an iterative training and optimization process of the Gurobi model. By analyzing the results over multiple iterations and adjusting these coefficients, one can refine the model's performance and achieve better task allocations that meet specific business objectives.

However, due to constraints in our current setup, we are unable to conduct this iterative training. As a result, for the purpose of this demonstration, we are using placeholder values for $\beta_1$ and $\beta_2$, both assumed to be 0.5. This means that, in our current model, both parts of the objective function are given equal importance. 

3. **Constraints**
- One ad should only be allocated 1 moderator
- The total tasks allocated to each moderator cannot exceed their max_tasks_per_day
- The delivery_country of the ad must match the market of the moderator
- If moderator's category expertise is not null, expertise must match category of ad
- If null, each moderator can only be assigned a maximum of 3 ad categories a day
    - This is to ensure that the moderator's work is more focused to improve productivity

In [271]:
from gurobipy import Model, GRB

# Initialize the Gurobi model
m = Model("AdTaskAllocation")

# Create the decision variables
x = {(ad_id, mod): m.addVar(vtype=GRB.BINARY, name=f"x_{ad_id}_{mod}")
     for ad_id in ads_dataset['ad_id']
     for mod in moderator_data['moderator']}

In [272]:
# Set the objective function
m.setObjective(sum(x[ad_row['ad_id'], mod_row['moderator']] * abs(0.5 * (ad_row['ad_score'] - mod_row['moderator_score']) + 
    0.5 * (ad_row['confidence'] - mod_row['normalized_productivity'] + mod_row['normalized_accuracy'])) 
                for _, ad_row in ads_dataset.iterrows() for _, mod_row in moderator_data.iterrows()), GRB.MINIMIZE)

In [273]:
# Constraints 1-5

# Each ad should be allocated to only one moderator
for _, ad_row in ads_dataset.iterrows():
    ad_id = ad_row['ad_id']
    m.addConstr(sum(x[ad_id, mod_row['moderator']] for _, mod_row in moderator_data.iterrows()) == 1)

# The total tasks assigned to a moderator should not exceed their max tasks per day
for _, mod_row in moderator_data.iterrows():
    mod = mod_row['moderator']
    m.addConstr(sum(x[ad_row['ad_id'], mod] for _, ad_row in ads_dataset.iterrows()) <= mod_row['max_tasks_per_day'])

# Only assign an ad to a moderator if the ad's market matches the moderator's market

# Preprocess moderator data into a dictionary by market
moderator_market_dict = {}
for _, mod_row in moderator_data.iterrows():
    mod = mod_row['moderator']
    market = eval(mod_row['market'])
    for i in market:
        if i not in moderator_market_dict:
            moderator_market_dict[i] = []
        moderator_market_dict[i].append(mod)

for _, ad_row in ads_dataset.iterrows():
    ad_id = ad_row['ad_id']
    ad_market = ad_row['delivery_country']

    # Get the list of moderators that match the ad's market from the preprocessed dictionary
    matching_mods = moderator_market_dict.get(ad_market, [])

    # Get all the moderators
    all_mods = moderator_data['moderator'].tolist()

    # For each moderator, if they are in the matching moderators list,
    # set their assignment variable for the current ad to 1. Otherwise, set it to 0.
    for mod in all_mods:
        if mod in matching_mods:
            m.addConstr(x[ad_id, mod] <= 1)  # Allow matching moderators to be assigned the ad
        else:
            m.addConstr(x[ad_id, mod] == 0)  # Non-matching moderators cannot be assigned the ad

# Extract unique ad categories
unique_categories = ads_dataset['category'].unique()

# Constraint 4
y = {}  # Store the binary variable y_{moderator, category}
for _, mod_row in moderator_data.iterrows():
    expertise = eval(mod_row['expertise'])
    if not expertise:  # If the expertise list is empty
        for category in unique_categories:
            y[mod_row['moderator'], category] = m.addVar(vtype=GRB.BINARY, name=f"y_{mod_row['moderator']}_{category}")
            
            # If the moderator is assigned at least one ad of that category, y should be 1
            m.addConstr(sum(x[ad_row['ad_id'], mod_row['moderator']] for _, ad_row in ads_dataset[ads_dataset['category'] == category].iterrows()) >= y[mod_row['moderator'], category])
        
        # Sum of all y for this moderator should be <= 3
        m.addConstr(sum(y[mod_row['moderator'], category] for category in unique_categories) <= 3)

# Constraint 5
for _, mod_row in moderator_data.iterrows():
    expertise = eval(mod_row['expertise'])
    if expertise:  # If the expertise list is not empty
        for _, ad_row in ads_dataset.iterrows():
            if ad_row['category'] not in expertise:
                m.addConstr(x[ad_row['ad_id'], mod_row['moderator']] == 0)  # This ad should not be assigned to this

In [ ]:
# Solve the model
m.optimize()

In [ ]:
# Extract the assignments from the solution
assignments = {}
for _, ad_row in ads_dataset.iterrows():
    for _, mod_row in moderator_data.iterrows():
        if x[ad_row['ad_id'], mod_row['moderator']].x > 0.5:  # If this ad is assigned to this moderator
            assignments[ad_row['ad_id']] = mod_row['moderator']

assignments

In [267]:
# Convert the dictionary of assignments into a DataFrame
assignments_df = pd.DataFrame(list(assignments.items()), columns=['ad_id', 'moderator'])

# Count the number of ads assigned to each moderator
assigned_counts = assignments_df.groupby('moderator').size().reset_index(name='num_ads_assigned')

# Merge with the original moderator_data
merged_data = pd.merge(moderator_data, assigned_counts, left_on='moderator', right_on='moderator', how='left')

# Fill NaN values with 0 (for moderators with no assignments)
merged_data['num_ads_assigned'].fillna(0, inplace=True)

# Calculate the increase in utilization % and new utilization % for each moderator
merged_data['increase_in_utilisation'] = (merged_data['num_ads_assigned'] * merged_data['handling time']) / (8 * 60 * 60 * 1000)
merged_data['new_utilisation'] = merged_data['Utilisation %'] + merged_data['increase_in_utilisation']

In [268]:
# Merge the ads dataset with the assignments dataframe to include the moderator assigned to each ad
ads_with_moderator = pd.merge(ads_dataset, assignments_df, on='ad_id', how='left')

# Fill NaN values with "Not Assigned" (for ads with no assignments)
ads_with_moderator['moderator'].fillna("Not Assigned", inplace=True)